In [86]:
class Directory:
    def __init__(self, name, parent=None, depth=0):
        self.name = name
        self.files = {}
        self.dirs = {}
        self.depth = depth
        self.parent = parent
        self.total_size = None
    def calc_total_size(self):
        if self.total_size is not None:
            return self.total_size
        self.total_size = sum([fs for fs in self.files.values()])
        for d in self.dirs.values():
            self.total_size += d.calc_total_size()
        return self.total_size
    
    def __str__(self):
        retstr = "  " * self.depth + "- " + self.name + "(dir) \n"
        for d in self.dirs.values():
            retstr += str(d) + "\n" 
        for k,v in self.files.items():
            retstr += "  " * self.depth + "  - " + k + " (file, size=" + str(v) + ")\n"
        return retstr

         


In [87]:
example = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
"""

In [88]:
def parse(s: str):
    root = Directory('/')
    current = root
    for line in s.splitlines():
        if line.startswith('$'):
            if len(line.split()) < 3:
                # must have been an ls
                assert(line.split()[1] == "ls")
                continue
            _, command, arg = line.split()
            if command == "cd":
                if arg == "..":
                    current = current.parent
                elif arg == "/":
                    current = root
                else:
                    if arg not in current.dirs:
                        current.dirs[arg] = Directory(arg, current, current.depth + 1)
                    current = current.dirs[arg]
        else:
            part1, part2 = line.split()
            if part1 == "dir":
                if part2 not in current.dirs:
                    current.dirs[part2] = Directory(part2, current, current.depth + 1)
            else:
                name, size = part2, part1
                current.files[name] = int(size)
    return root

In [91]:
def part1r(d: Directory):
    dir_sizes = []
    for child in d.dirs.values():
        dir_sizes = dir_sizes + part1r(child)
    dir_sizes = [d.calc_total_size()] + dir_sizes
    return dir_sizes
    

In [97]:
def part1(dir):
    # print(dir)
    return sum([size for size in part1r(dir) if size <= 100000])
    
part1(parse(open("../inputs/07.txt").read()))

1778099

In [105]:
def part2(dir):
    sizes = sorted(part1r(dir), reverse=True)
    free = 70000000 - sizes[0]
    required = 30000000
    for i in reversed(sizes):
        if free + i >= required:
            return i


part2(parse(open("../inputs/07.txt").read()))

1623571